In [1]:
import os, rasterio
import numpy as np
from rasterio.transform import Affine
os.getcwd()

'C:\\Users\\erfan\\Dropbox (UNC Charlotte)\\Researches\\Erfan Dissertation\\Scripts\\py\\SUM_Charlotte'

In [2]:
# ------------------------------------------
# INPUT / OUTPUT PATHS
# ------------------------------------------

# Path to your DEM (can be the clipped DEM or the original meck tif)
dem_path = "../../../Data/Original_dataset/Archive/DEM/mecklenburg-DEM03/mecklenburg-DEM03.tif"

# Output slope raster (in percent)
slope_output = "../../../Data/Original_dataset/Archive/DEM/meck_slope_percent.tif"

In [3]:
# ------------------------------------------
# 1) READ DEM
# ------------------------------------------
with rasterio.open(dem_path) as src:
    dem = src.read(1).astype("float32")   # Read band 1 of the DEM as a float array
    transform = src.transform             # Affine transform (contains pixel size & origin)
    meta = src.meta.copy()                # Copy metadata for writing output raster

In [4]:
# ------------------------------------------
# 2) COMPUTE SLOPE USING 3x3 SOBEL OPERATOR
# ------------------------------------------

# Sobel / Horn gradient kernels
kernel_x = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
], dtype="float32")

kernel_y = np.array([
    [1, 2, 1],
    [0, 0, 0],
    [-1, -2, -1]
], dtype="float32")

# Pad DEM to avoid edge issues
dem_padded = np.pad(dem, 1, mode='edge')

# Gradient arrays
dzdx = np.zeros_like(dem, dtype="float32")
dzdy = np.zeros_like(dem, dtype="float32")

rows, cols = dem.shape

# Apply convolution manually for speed + portability
for i in range(rows):
    for j in range(cols):
        window = dem_padded[i:i + 3, j:j + 3]
        dzdx[i, j] = np.sum(kernel_x * window) / (8 * dx)
        dzdy[i, j] = np.sum(kernel_y * window) / (8 * dy)

# Compute slope in percent rise
slope_radians = np.sqrt(dzdx ** 2 + dzdy ** 2)
slope_percent = slope_radians * 100.0

# Handle nodata if present
if nodata is not None:
    slope_percent[dem == nodata] = nodata

MemoryError: Unable to allocate 11.1 GiB for an array with shape (60000, 49600) and data type float32

In [ ]:
# ------------------------------------------
# 3) WRITE OUTPUT RASTER
# ------------------------------------------
meta.update({
    "dtype": "float32",
    "count": 1,
    "nodata": nodata
})

with rasterio.open(slope_output, "w", **meta) as dst:
    dst.write(slope_percent.astype("float32"), 1)

print("Slope raster saved:", slope_output)
